In [ ]:
import pandas as pd
import unicodedata
import re, math, random, os
from torch.utils.data import Dataset
import torch
rnn_encoder, rnn_encoder, transformer_encoder, transformer_decoder = None, None, None, None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__=='__main__':
    print('Using device:', DEVICE)

Using device: cuda


# Helper Functions
This cell contains helper functions for the notebook.

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    """Normalizes latin chars with accent to their canonical decomposition"""
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    '''
    Preprocess the sentence to add the start, end tokens and make them lower-case
    '''
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r'([?.!,¿])', r' \1 ', w)
    w = re.sub(r'[" "]+', ' ', w)

    w = re.sub(r'[^a-zA-Z?.!,¿]+', ' ', w)

    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w


def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len:
        padded[:] = x[:max_len]
    else:
        padded[:len(x)] = x
    return padded


def preprocess_data_to_tensor(dataframe, src_vocab, trg_vocab):
    # Vectorize the input and target languages
    src_tensor = [[src_vocab.word2idx[s if s in src_vocab.vocab else '<unk>'] for s in es.split(' ')] for es in dataframe['es'].values.tolist()]
    trg_tensor = [[trg_vocab.word2idx[s if s in trg_vocab.vocab else '<unk>'] for s in eng.split(' ')] for eng in dataframe['eng'].values.tolist()]

    # Calculate the max_length of input and output tensor for padding
    max_length_src, max_length_trg = max(len(t) for t in src_tensor), max(len(t) for t in trg_tensor)
    print('max_length_src: {}, max_length_trg: {}'.format(max_length_src, max_length_trg))

    # Pad all the sentences in the dataset with the max_length
    src_tensor = [pad_sequences(x, max_length_src) for x in src_tensor]
    trg_tensor = [pad_sequences(x, max_length_trg) for x in trg_tensor]

    return src_tensor, trg_tensor, max_length_src, max_length_trg


def train_test_split(src_tensor, trg_tensor):
    '''
    Create training and test sets.
    '''
    total_num_examples = len(src_tensor) - int(0.2*len(src_tensor))
    src_tensor_train, src_tensor_test = src_tensor[:int(0.75*total_num_examples)], src_tensor[int(0.75*total_num_examples):total_num_examples]
    trg_tensor_train, trg_tensor_test = trg_tensor[:int(0.75*total_num_examples)], trg_tensor[int(0.75*total_num_examples):total_num_examples]

    return src_tensor_train, src_tensor_test, trg_tensor_train, trg_tensor_test

## Sanity Check Function

The code below will be used to perform a sanity check on both the RNN and transformer later in the homework

In [ ]:
count_parameters = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

def sanityCheckModel(all_test_params, NN, expected_outputs, init_or_forward):
    print('--- TEST: ' + ('Number of Model Parameters (tests __init__(...))' if init_or_forward=='init' else 'Output shape of forward(...)') + ' ---')
    if init_or_forward == "forward":
        # Creating random texts and lables batches
        texts_batch = torch.randint(low=0, high=len(all_test_params[0]['src_vocab']), size=(10,16))
        labels_batch = torch.randint(low=0, high=len(all_test_params[0]['src_vocab']), size=(10,12))

    for tp_idx, (test_params, expected_output) in enumerate(zip(all_test_params, expected_outputs)):
        if init_or_forward == "forward":
            batch_size = test_params['batch_size']
            texts = texts_batch[:batch_size]
            # if NN.__name__ == "RnnEncoder":
            texts = texts.transpose(0,1)

        # Construct the student model
        tps = {k:v for k, v in test_params.items() if k != 'batch_size'}
        stu_nn = NN(**tps)

        input_rep = str({k:v for k,v in tps.items()})

        if init_or_forward == "forward":
            with torch.no_grad():
                if NN.__name__ == "TransformerEncoder":
                    stu_out = stu_nn(texts)
                else:
                    stu_out, _ = stu_nn(texts)
                    expected_output = torch.rand(expected_output).size()
            ref_out_shape = expected_output

            has_passed = torch.is_tensor(stu_out)
            if not has_passed: msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
            else:
                has_passed = stu_out.shape == ref_out_shape
                msg = 'Your Output Shape: ' + str(stu_out.shape)


            status = 'PASSED' if has_passed else 'FAILED'
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape: ' + str(texts.shape) + '\tExpected Output Shape: ' + str(ref_out_shape) + '\t' + msg
            print(message)
        else:
            stu_num_params = count_parameters(stu_nn)
            ref_num_params = expected_output
            comparison_result = (stu_num_params == ref_num_params)

            status = 'PASSED' if comparison_result else 'FAILED'
            message = '\t' + status + "\tInput: " + input_rep + ('\tExpected Num. Params: ' + str(ref_num_params) + '\tYour Num. Params: '+ str(stu_num_params))
            print(message)

        del stu_nn

## Evaluation Functions

These functions will be used to evaluate both the RNN and Transformer Models.

In [ ]:
def get_reference_candidate(target, pred, trg_vocab):
    def _to_token(sentence):
        lis = []
        for s in sentence[1:]:
            x = trg_vocab.idx2word[s]
            if x == "<end>": break
            lis.append(x)
        return lis
    reference = _to_token(target)
    candidate = _to_token(pred)
    return reference, candidate

def compute_bleu_scores(target_output, final_output, trg_vocab):
    assert len(target_output) == len(final_output)
    bleu_1 = 0.0
    bleu_2 = 0.0
    bleu_3 = 0.0
    bleu_4 = 0.0

    smoother = SmoothingFunction()
    save_reference = []
    save_candidate = []
    for i in range(len(target_output)):
        reference, candidate = get_reference_candidate(target_output[i], final_output[i], trg_vocab)

        bleu_1 += sentence_bleu(reference, candidate, weights=(1,), smoothing_function=smoother.method1)
        bleu_2 += sentence_bleu(reference, candidate, weights=(1/2, 1/2), smoothing_function=smoother.method1)
        bleu_3 += sentence_bleu(reference, candidate, weights=(1/3, 1/3, 1/3), smoothing_function=smoother.method1)
        bleu_4 += sentence_bleu(reference, candidate, weights=(1/4, 1/4, 1/4, 1/4), smoothing_function=smoother.method1)

        save_reference.append(reference)
        save_candidate.append(candidate)

    bleu_1 = bleu_1/len(target_output)
    bleu_2 = bleu_2/len(target_output)
    bleu_3 = bleu_3/len(target_output)
    bleu_4 = bleu_4/len(target_output)

    scores = {"bleu_1": bleu_1, "bleu_2": bleu_2, "bleu_3": bleu_3, "bleu_4": bleu_4}
    print('BLEU 1-gram: %f' % (bleu_1))
    print('BLEU 2-gram: %f' % (bleu_2))
    print('BLEU 3-gram: %f' % (bleu_3))
    print('BLEU 4-gram: %f' % (bleu_4))

    return save_candidate, scores

# Step 1: Download & Prepare the Data

## Download and Visualize the Data

Here we will download the translation data. We will learn a model to translate Spanish to English.

In [ ]:
if __name__ == '__main__':
    os.system("wget http://www.manythings.org/anki/spa-eng.zip")
    os.system("unzip -o spa-eng.zip")

Now we view the data.

In [ ]:
if __name__ == '__main__':

    total_num_examples = 50000
    dat = pd.read_csv("spa.txt",
                    sep="\t",
                    header=None,
                    usecols=[0,1],
                    names=['eng', 'es'],
                    nrows=total_num_examples,
                    encoding="UTF-8"
    ).sample(frac=1).reset_index().drop(['index'], axis=1)

    print(dat) # Visualize the data

                          eng                             es
0           I bumped my knee.          Me golpeé la rodilla.
1           I'll be at Tom's.         Estaré en casa de Tom.
2       We love our children.       Amamos a nuestros hijos.
3        He took out one egg.              Él sacó un huevo.
4       He achieved his goal.            Él alcanzó su meta.
...                       ...                            ...
49995    The door burst open.   La puerta se abrió de golpe.
49996  She never wears green.  Ella nunca se viste de verde.
49997   Those books are mine.                Son mis libros.
49998  I don't play baseball.           No juego al béisbol.
49999    Please stop the war.    Paren la guerra, por favor.

[50000 rows x 2 columns]


Next we preprocess the data.

In [ ]:
if __name__ == '__main__':
    data = dat.copy()
    data['eng'] = dat.eng.apply(lambda w: preprocess_sentence(w))
    data['es'] = dat.es.apply(lambda w: preprocess_sentence(w))
    print(data) # Visualizing the data

                                         eng  \
0           <start> i bumped my knee . <end>   
1           <start> i ll be at tom s . <end>   
2       <start> we love our children . <end>   
3        <start> he took out one egg . <end>   
4       <start> he achieved his goal . <end>   
...                                      ...   
49995    <start> the door burst open . <end>   
49996  <start> she never wears green . <end>   
49997   <start> those books are mine . <end>   
49998  <start> i don t play baseball . <end>   
49999    <start> please stop the war . <end>   

                                                 es  
0              <start> me golpee la rodilla . <end>  
1             <start> estare en casa de tom . <end>  
2           <start> amamos a nuestros hijos . <end>  
3                  <start> el saco un huevo . <end>  
4                <start> el alcanzo su meta . <end>  
...                                             ...  
49995   <start> la puerta se abrio de golpe .

## Vocabulary & Dataloader

First we create a class for managing our vocabulary as we did in Homework 2. In this homework, we have a separate class for the vocabulary as we need 2 different vocabularies $-$ one for English and one for Spanish.

Then we prepare the dataloader and make sure it returns the source sentence and target sentence.

In [ ]:
class Vocab_Lang():
    def __init__(self, vocab):
        self.word2idx = {'<pad>': 0, '<unk>': 1}
        self.idx2word = {0: '<pad>', 1: '<unk>'}
        self.vocab = vocab

        for index, word in enumerate(vocab):
            self.word2idx[word] = index + 2 # +2 because of <pad> and <unk> token
            self.idx2word[index + 2] = word

    def __len__(self):
        return len(self.word2idx)

    def __repr__(self):
        if len(self.vocab) <= 5:
            return str(self.vocab)
        else:
            return f'Vocab_Lang object with {len(self.vocab)} words'

class MyData(Dataset):
    def __init__(self, X, y):
        self.length = torch.LongTensor([np.sum(1 - np.equal(x, 0)) for x in X])
        self.data = torch.LongTensor(X)
        self.target = torch.LongTensor(y)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
import numpy as np
import random
from torch.utils.data import DataLoader

In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS
    BATCH_SIZE = 64
    EMBEDDING_DIM = 256

In [ ]:
def build_vocabulary(pd_dataframe):
    sentences = [sen.split() for sen in pd_dataframe]
    vocab = {}
    for sen in sentences:
        for word in sen:
            if word not in vocab:
                vocab[word] = 1
    return list(vocab.keys())

if __name__ == '__main__':
    src_vocab_list = build_vocabulary(data['es'])
    trg_vocab_list = build_vocabulary(data['eng'])

We instantiate our training and validation datasets.

In [ ]:
if __name__ == '__main__':
    src_vocab = Vocab_Lang(src_vocab_list)
    trg_vocab = Vocab_Lang(trg_vocab_list)

    src_tensor, trg_tensor, max_length_src, max_length_trg = preprocess_data_to_tensor(data, src_vocab, trg_vocab)
    src_tensor_train, src_tensor_val, trg_tensor_train, trg_tensor_val = train_test_split(src_tensor, trg_tensor)

    # Create train and val datasets
    train_dataset = MyData(src_tensor_train, trg_tensor_train)
    train_dataset = DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)

    test_dataset = MyData(src_tensor_val, trg_tensor_val)
    test_dataset = DataLoader(test_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=False)

max_length_src: 16, max_length_trg: 12


In [ ]:
if __name__ == '__main__':
    idxes = random.choices(range(len(train_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    print('Source:', src)
    print('Source Dimensions: ', src.size())
    print('Target:', trg)
    print('Target Dimensions: ', trg.size())

Source: tensor([[   2,   64, 1407,    7,    8,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2,   38,    3,   28,   90,  308,    7,    8,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2, 2879,  207, 1127, 8790,    7,    8,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2, 2748,  568,    7,    8,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [   2,  435,   30,  237,    7,    8,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
Source Dimensions:  torch.Size([5, 16])
Target: tensor([[   2,    3,  900,   36,    7,    8,    0,    0,    0,    0,    0,    0],
        [   2,    3,  127,   46,  192,    5,  262,    7,    8,    0,    0,    0],
        [   2,  358,   64,  887,   82,    7,    8,    0,    0,    0,    0,    0],
        [   2,    3, 1154,  473,    7,    8,    0,    0,    0,    0,    0,    0],
        [   2,    3,  364,  208,    7,  

# Step 2: Train a Recurrent Neural Network (RNN)

Writing a recurrent model for machine translation, and then trainining and evaluating its results.

1. Attention paper: https://arxiv.org/pdf/1409.0473.pdf
2. Explanation of LSTM's & GRU's: https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
3. Attention explanation: https://towardsdatascience.com/attention-in-neural-networks-e66920838742
4. Another attention explanation: https://towardsdatascience.com/attention-and-its-different-forms-7fc3674d14dc


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import time
from tqdm.notebook import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction, corpus_bleu

##  Encoder Model

Building a recurrent encoder model, instead of using a fully connected layer as the output, returns a sequence of outputs of GRU as well as the final hidden state. These will be used in the decoder.

In [ ]:
class RnnEncoder(nn.Module):
    def __init__(self, src_vocab, embedding_dim, hidden_units):
        super(RnnEncoder, self).__init__()
        """
        Args:
            src_vocab: Vocab_Lang, the source vocabulary
            embedding_dim: the dimension of the embedding
            hidden_units: The number of features in the GRU hidden state
        """
        self.src_vocab = src_vocab # Do not change
        vocab_size = len(src_vocab)


        # Initialize embedding layer
        # (see: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

        # Initialize a single directional GRU with 1 layer and batch_first=False

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.gru = nn.GRU(embedding_dim, hidden_units, num_layers=1, batch_first=False)

    def forward(self, x):
        """
        Args:
            x: source texts, [max_len, batch_size]

        Returns:
            output: [max_len, batch_size, hidden_units]
            hidden_state: [1, batch_size, hidden_units]

        Pseudo-code:
        - Pass x through an embedding layer and pass the results through the recurrent net
        - Return output and hidden states from the recurrent net
        """
        #output, hidden_state = None, None

        x_embedded = self.embedding(x)

        # Pass through GRU
        output, hidden_state = self.gru(x_embedded)

        return output, hidden_state

In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [8110, 31210, 122410, 8575, 32125, 124225, 9040, 33040, 126040]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2]
    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            inp['embedding_dim'] = EMBEDDING_DIM
            inp['src_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 50]), torch.Size([16, 2, 50]), torch.Size([16, 1, 100]), torch.Size([16, 2, 100]), torch.Size([16, 1, 200]), torch.Size([16, 2, 200])]

    sanityCheckModel(inputs, RnnEncoder, expected_outputs, "forward")

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 50}	Expected Num. Params: 8110	Your Num. Params: 8110
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 100}	Expected Num. Params: 31210	Your Num. Params: 31210
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 200}	Expected Num. Params: 122410	Your Num. Params: 122410
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 50}	Expected Num. Params: 8575	Your Num. Params: 8575
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 100}	Expected Num. Params: 32125	Your Num. Params: 32125
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 200}	Expected Num. Params: 124225	Your Num. Params: 124225
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'hidden_units': 50

We will implement a Decoder model that uses an attention mechanism, as provided in https://arxiv.org/pdf/1409.0473.pdf.

  1. <b>Attention scores:</b> Compute real-valued scores for the decoder hidden state $\mathbf{h}_t$ and each encoder hidden state $\mathbf{\bar h}_s$: $$\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_s)=
      \mathbf{v}_a^T \tanh(\mathbf{W}_1 \mathbf{h}_t +\mathbf{W}_2 \mathbf{\bar h}_s)
$$
  A higher score indicates a stronger "affinity" between the decoder state and a specific encoder state.

 2. <b>Attention weights:</b> Normalizing the attention scores to obtain a valid probability distribution: $$\alpha_{ts} = \frac{\exp \big (\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_s) \big)}{\sum_{s'=1}^S \exp \big (\mathrm{score}(\mathbf{h}_t, \mathbf{\bar h}_{s'}) \big)}$$ (softmax function)

 3. <b>Context vector:</b>  $\mathbf{c}_t$  is a weighted average of the encoder hidden states, where the weights are given by the attention weights  just computed: $$\mathbf{c}_t=\sum_{s=1}^S \alpha_{ts} \mathbf{\bar h}_s$$

Returns this context vector, along with the attention weights.


In [ ]:
class RnnDecoder(nn.Module):
    def __init__(self, trg_vocab, embedding_dim, hidden_units):
        super(RnnDecoder, self).__init__()
        """
        Args:
            trg_vocab: Vocab_Lang, the target vocabulary
            embedding_dim: The dimension of the embedding
            hidden_units: The number of features in the GRU hidden state
        """
        self.trg_vocab = trg_vocab # Do not change
        vocab_size = len(trg_vocab)


        # Initialize embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Initialize layers to compute attention score
        self.W1 = nn.Linear(hidden_units, hidden_units)
        self.W2 = nn.Linear(hidden_units, hidden_units)
        self.Va = nn.Linear(hidden_units, 1)

        # Initialize a single directional GRU with 1 layer and batch_first=True
        # NOTE: Input to your RNN will be the concatenation of your embedding vector and the context vector
        self.gru = nn.GRU(embedding_dim + hidden_units, hidden_units, num_layers=1, batch_first=True)


        # Initialize fully connected layer
        self.fc = nn.Linear(hidden_units, vocab_size)

    def compute_attention(self, dec_hs, enc_output):
        '''
        This function computes the context vector and attention weights.

        Args:
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]

        Returns:
            context_vector: Context vector, according to formula; [batch_size, hidden_units]
            attention_weights: The attention weights you have calculated; [batch_size, max_len_src, 1]

        Pseudo-code:
            (1) Compute the attention scores for dec_hs & enc_output
                    - Hint: You may need to permute the dimensions of the tensors in order to pass them through linear layers
                    - Output size: [batch_size, max_len_src, 1]
            (2) Compute attention_weights by taking a softmax over your scores to normalize the distribution (Make sure that after softmax the normalized scores add up to 1)
                    - Output size: [batch_size, max_len_src, 1]
            (3) Compute context_vector from attention_weights & enc_output
                    - Hint: You may find it helpful to use torch.sum & element-wise multiplication (* operator)
            (4) Return context_vector & attention_weights
        '''
        context_vector, attention_weights = None, None


        dec_hs = dec_hs.permute(1, 0, 2)
        enc_output = enc_output.permute(1, 0, 2)

        score = self.Va(torch.tanh(self.W1(enc_output) + self.W2(dec_hs)))

        attention_weights = F.softmax(score, dim=1)

        context_vector = torch.sum(attention_weights * enc_output, dim=1)

        return context_vector, attention_weights

    def forward(self, x, dec_hs, enc_output):
        '''
        This function runs the decoder for a **single** time step.

        Args:
            x: Input token; [batch_size, 1]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
            attention_weights: The attention weights you have learned; [batch_size, max_len_src, 1]

        Pseudo-code:
            (1) Compute the context vector & attention weights by calling self.compute_attention(...) on the appropriate input
            (2) Obtain embedding vectors for your input x
                    - Output size: [batch_size, 1, embedding_dim]
            (3) Concatenate the context vector & the embedding vectors along the appropriate dimension
            (4) Feed this result through your RNN (along with the current hidden state) to get output and new hidden state
                    - Output sizes: [batch_size, 1, hidden_units] & [1, batch_size, hidden_units]
            (5) Feed the output of your RNN through linear layer to get (unnormalized) output distribution (don't call softmax!)
            (6) Return this output, the new decoder hidden state, & the attention weights
        '''
        fc_out, attention_weights = None, None

        context_vector, attention_weights = self.compute_attention(dec_hs, enc_output)
        x = self.embedding(x.type(torch.long))
        context_vector = context_vector.unsqueeze(1)
        rnn_input = torch.cat((x, context_vector), dim=-1)
        output, dec_hs = self.gru(rnn_input, dec_hs)
        fc_out = self.fc(output.squeeze(1))

        return fc_out, dec_hs, attention_weights

    def decode_step(self, inputs, enc_output, dec_hs):
        '''
        Call one step of the decoder.

        Args:
            inputs: Input tokens; [batch_size, sequence length]
            enc_output: Encoder outputs; [max_len_src, batch_size, hidden_units]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: Decoder hidden state; [1, batch_size, hidden_units]
        '''
        assert inputs.shape[0] == enc_output.shape[1] == dec_hs.shape[1], 'batch_size must be the same across tensors'
        fc_out, dec_hs, attention_weights = self(inputs[:,-1].unsqueeze(1), dec_hs, enc_output)
        return fc_out, dec_hs

In [ ]:
def sanityCheckDecoderModelForward(inputs, NN, expected_outputs):
    print('--- TEST: Output shape of forward(...) ---\n')
    expected_fc_outs = expected_outputs[0]
    expected_dec_hs = expected_outputs[1]
    expected_attention_weights = expected_outputs[2]
    msg = ''
    for i, inp in enumerate(inputs):
        input_rep = '{'
        for k,v in inp.items():
            if torch.is_tensor(v):
                input_rep += str(k) + ': ' + 'Tensor with shape ' + str(v.size()) + ', '
            else:
                input_rep += str(k) + ': ' + str(v) + ', '
        input_rep += '}'
        dec = RnnDecoder(trg_vocab=inp['trg_vocab'],embedding_dim=inp['embedding_dim'],hidden_units=inp['hidden_units'])
        dec_hs = torch.rand(1, inp["batch_size"], inp['hidden_units'])
        x = torch.randint(low=0,high=len(inp["trg_vocab"]),size=(inp["batch_size"], 1))
        with torch.no_grad():
            dec_out = dec(x=x, dec_hs=dec_hs,enc_output=inp['encoder_outputs'])
            if not isinstance(dec_out, tuple):
                msg = '\tFAILED\tYour RnnDecoder.forward() output must be a tuple; received ' + str(type(dec_out))
                print(msg)
                continue
            elif len(dec_out)!=3:
                msg = '\tFAILED\tYour RnnDecoder.forward() output must be a tuple of size 3; received tuple of size ' + str(len(dec_out))
                print(msg)
                continue
            stu_fc_out, stu_dec_hs, stu_attention_weights = dec_out
        del dec
        has_passed = True
        msg = ""
        if not torch.is_tensor(stu_fc_out):
            has_passed = False
            msg += '\tFAILED\tOutput must be a torch.Tensor; received ' + str(type(stu_fc_out)) + " "
        if not torch.is_tensor(stu_dec_hs):
            has_passed = False
            msg += '\tFAILED\tDecoder Hidden State must be a torch.Tensor; received ' + str(type(stu_dec_hs)) + " "
        if not torch.is_tensor(stu_attention_weights):
            has_passed = False
            msg += '\tFAILED\tAttention Weights must be a torch.Tensor; received ' + str(type(stu_attention_weights)) + " "

        status = 'PASSED' if has_passed else 'FAILED'
        if not has_passed:
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(os.XATTR_REPLACE.shape) + '\tExpected Output Shape: ' + str(expected_fc_outs[i]) + '\t' + msg
            print(message)
            continue

        has_passed = stu_fc_out.size() == expected_fc_outs[i]
        msg = 'Your Output Shape: ' + str(stu_fc_out.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Output Shape: ' + str(expected_fc_outs[i]) + '\t' + msg
        print(message)

        has_passed = stu_dec_hs.size() == expected_dec_hs[i]
        msg = 'Your Hidden State Shape: ' + str(stu_dec_hs.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Hidden State Shape: ' + str(expected_dec_hs[i]) + '\t' + msg
        print(message)

        has_passed = stu_attention_weights.size() == expected_attention_weights[i]
        msg = 'Your Attention Weights Shape: ' + str(stu_attention_weights.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (x): ' + str(x.shape) + '\tExpected Attention Weights Shape: ' + str(expected_attention_weights[i]) + '\t' + msg
        print(message)

        stu_sum = stu_attention_weights.sum(dim=1).squeeze()
        if torch.allclose(stu_sum, torch.ones_like(stu_sum), atol=1e-5):
            print('\tPASSED\t The sum of your attention_weights along dim 1 is 1.')
        else:
            print('\tFAILED\t The sum of your attention_weights along dim 1 is not 1.')
        print()

In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    embedding_dim = [2, 5, 8]
    hidden_units = [50, 100, 200]
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for hu in hidden_units:
            inp = {}
            inp['trg_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['hidden_units'] = hu
            inputs.append(inp)
    # Test init
    expected_outputs = [21016, 82016, 324016, 21481, 82931, 325831, 21946, 83846, 327646]
    sanityCheckModel(inputs, RnnDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    hidden_units = [50, 100, 200]
    batch_sizes = [1, 2, 4]
    embedding_dims = iter([50,80,100,120,150,200,300,400,500])
    encoder_outputs = iter([torch.rand([16, 1, 50]), torch.rand([16, 2, 50]), torch.rand([16, 4, 50]), torch.rand([16, 1, 100]), torch.rand([16, 2, 100]), torch.rand([16, 4, 100]), torch.rand([16, 1, 200]), torch.rand([16, 2, 200]),torch.rand([16, 4, 200])])
    expected_fc_outs = [torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5]),torch.Size([1, 5]),torch.Size([2, 5]),torch.Size([4, 5])]
    expected_dec_hs = [torch.Size([1, 1, 50]), torch.Size([1, 2, 50]), torch.Size([1, 4, 50]), torch.Size([1, 1, 100]), torch.Size([1, 2, 100]), torch.Size([1, 4, 100]), torch.Size([1, 1, 200]), torch.Size([1, 2, 200]), torch.Size([1, 4, 200])]
    expected_attention_weights = [torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1]), torch.Size([1, 16, 1]), torch.Size([2, 16, 1]), torch.Size([4, 16, 1])]
    expected_outputs = (expected_fc_outs, expected_dec_hs, expected_attention_weights)

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp["batch_size"] = b
            inp['hidden_units'] = hu
            inp['encoder_outputs'] = next(encoder_outputs)
            inputs.append(inp)

    sanityCheckDecoderModelForward(inputs, RnnDecoder, expected_outputs)


--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 50}	Expected Num. Params: 21016	Your Num. Params: 21016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 100}	Expected Num. Params: 82016	Your Num. Params: 82016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 2, 'hidden_units': 200}	Expected Num. Params: 324016	Your Num. Params: 324016
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 50}	Expected Num. Params: 21481	Your Num. Params: 21481
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 100}	Expected Num. Params: 82931	Your Num. Params: 82931
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 5, 'hidden_units': 200}	Expected Num. Params: 325831	Your Num. Params: 325831
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'hidden_units'

## Train RNN Model

We will train the encoder and decoder using cross-entropy loss.

In [ ]:

def loss_function(real, pred):
    mask = real.ge(1).float() # Only consider non-zero inputs in the loss

    loss_ = F.cross_entropy(pred, real) * mask
    return torch.mean(loss_)

def train_rnn_model(encoder, decoder, dataset, optimizer, trg_vocab, device, n_epochs):
    batch_size = dataset.batch_size
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0

        encoder.train()
        decoder.train()

        for src, trg in tqdm(dataset):
            n_batch += 1
            loss = 0

            enc_output, enc_hidden = encoder(src.transpose(0,1).to(device))
            dec_hidden = enc_hidden

            # use teacher forcing - feeding the target as the next input (via dec_input)
            dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size)

            # run code below for every timestep in the ys batch
            for t in range(1, trg.size(1)):
                predictions, dec_hidden, _ = decoder(dec_input.to(device), dec_hidden.to(device), enc_output.to(device))
                assert len(predictions.shape) == 2 and predictions.shape[0] == dec_input.shape[0] and predictions.shape[1] == len(trg_vocab.word2idx), "First output of decoder must have shape [batch_size, vocab_size], you returned shape " + str(predictions.shape)
                loss += loss_function(trg[:, t].to(device), predictions.to(device))
                dec_input = trg[:, t].unsqueeze(1)

            batch_loss = (loss / int(trg.size(1)))
            total_loss += batch_loss.item()

            optimizer.zero_grad()

            batch_loss.backward()

            ### update model parameters
            optimizer.step()

        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

    print('Model trained!')

In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS
    LEARNING_RATE = 0.001
    HIDDEN_UNITS=256
    N_EPOCHS=10

    rnn_encoder = RnnEncoder(src_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)
    rnn_decoder = RnnDecoder(trg_vocab, EMBEDDING_DIM, HIDDEN_UNITS).to(DEVICE)

    rnn_model_params = list(rnn_encoder.parameters()) + list(rnn_decoder.parameters())
    optimizer = torch.optim.Adam(rnn_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

Encoder and Decoder models initialized!


In [ ]:
if __name__ == '__main__':
    train_rnn_model(rnn_encoder, rnn_decoder, train_dataset, optimizer, trg_vocab, DEVICE, N_EPOCHS)

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 1/10	 Loss: 1.7197 	(16.17s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 2/10	 Loss: 1.0464 	(17.23s)


  0%|          | 0/468 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss: 0.1363 	(17.73s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 9/10	 Loss: 0.1038 	(17.43s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch:10/10	 Loss: 0.0832 	(17.85s)
Model trained!


# Step 3: Inference Functions

Now that we have trained the model, we can use it on test data. But first, I must *decode* from the model: given an input (Spanish sentence), what is the most likely output (English sentence)?

Recall that sequence-to-sequence models like RNN's factorize the probability distribution of output sequence $\mathbf{y}$ given input sequence $\mathbf{x}$ as $$P(\mathbf{y} \mid \mathbf{x})=\prod _{t=1}^{|\mathbf{y}|}P(y_t\mid y_0 \cdots y_{t-1}, \mathbf{x})$$

where $y_0$ is the start-of-sentence token.

First, y exploring how to *sample* from this distribution. Then, implementing a decoding method, which aims to find $\mathrm{argmax}_\mathbf{y} P(\mathbf{y} \mid \mathbf{x})$. Because there are infinitely many sequences $\mathbf{y}$, it is not possible to directly optimize this quantity; thus we resort to *beam search decoding*, a heuristic method.

## Temperature Sampling

Temperature sampling is a method to sample from $P(\mathbf{y}\mid \mathbf{x})$. At each timestep, the softmax function $\sigma(\mathbf{z})$ transforms the output logits $\mathbf{z}$ of the final layer of the transformer to a probability distribution. To apply temperature $T$, we use a modified softmax function to produce the distribution: $$P(z_i)=\frac{\exp (z_i\,/\,T)}{\sum _j \exp (z_j\,/\,T)}$$ What is the effect of applying temperature?
* If $T=1$, then this is equivalent to the softmax function $-$ hence, the unmodified probability distributon learned by the transformer.
* As $T \rightarrow \infty$, this approaches the uniform distribution.
* As $T \rightarrow 0$, the distribution becomes increasingly "peaked": that is, the probability of the most likely token increases to $1$.

Temperature sampling is implemented in the following cell

In [ ]:
def sample_model(encoder, decoder, src, max_decode_len, temperature):
    """
    Perform temperature sampling of the target sentence for each source sentence in src based on the trained encoder and decoder.

    Args:
        encoder: Your RnnEncoder object
        decoder: Your RnnDecoder object
        src: [max_src_length, batch_size] the source sentences you wish to translate
        max_decode_len: The maximum desired length (int) of your target translated sentences
        temperature: The temperature in temperature sampling

    Returns:
        sentences: A list of length batch_size, where sentences[i] is a list containing vocab indexes of the sampled
            trg sentence for the ith item in the src batch.

    Pseudo-code:
    - Obtain encoder output and hidden state by encoding src sentences
    - For 1 ≤ t < max_decode_len:
        - Obtain dec_input as the best words so far for previous time steps (you can get this from curr_output)
        - Obtain your prediction logits and hidden state by feeding dec_input, enc_output, and dec_hidden to decoder
        - Sample from the distribution over tokens (consider using torch.distributions.Categorical) with temperature
        - Save result in curr_output at index t
    - Truncate each sentence in the batch at EOS before returning them as a list of lists of vocab indexes


    """

    # Initialize variables
    trg_vocab = decoder.trg_vocab
    batch_size = src.size(1)
    curr_output = torch.zeros((batch_size, max_decode_len))
    eos_idx = trg_vocab.word2idx['<end>']
    device = src.device
    is_rnn = 'Rnn' in encoder.__class__.__name__
    sentences = []

    # We start the decoding with the start token for each example
    dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size, device=device)
    curr_output[:, 0] = dec_input.squeeze(1)

    # Obtain encoder outputs
    if is_rnn: enc_output, dec_hidden = encoder(src)
    else: enc_output, dec_hidden = encoder(src), None
    enc_output, dec_hidden = enc_output.to(device), dec_hidden.to(device) if dec_hidden is not None else None

    #   At each time step, sample the next token and save it in curr_output
    curr_output = curr_output.long()
    enc_output = enc_output.to(device)
    if dec_hidden is not None:
        dec_hidden = dec_hidden.to(device)

    for t in range(1, max_decode_len):
        logits, dec_hidden = decoder.decode_step(curr_output[:, :t].to(device), enc_output, dec_hidden)
        probs = F.softmax(logits/temperature, dim=-1)  # [batch_size, vocab_size]
        next_token = torch.distributions.Categorical(probs).sample()

        curr_output[:, t] = next_token


    #   For each item in the batch, truncate the sentence to EOS; if EOS was not predicted, choose sentence up to max_decode_len
    for i in range(batch_size):
        sentence = curr_output[i].tolist()
        if eos_idx in sentence:
            eos_pos = sentence.index(eos_idx)
            sentences.append(sentence[:eos_pos+1])
        else:
            sentence = sentence[:max_decode_len]
            if sentence[-1] != eos_idx:
                sentence.append(eos_idx)
            sentences.append(sentence)

    return sentences

Qualitatively compare some of the sentences generated by model with the some of the correct translations.

In [ ]:
if __name__ == '__main__':
    rnn_encoder.eval()
    rnn_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    results = {}
    for temp in [0.01, 1.0, 2.0]:
        results[temp] = sample_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), temp)
    for i in range(len(src)):
        print("Source sentence:\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        for temp in results:
            print("Sampled sentence (T="+str(temp)+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in results[temp][i]] if x != '<pad>']))
        print("----------------")

Source sentence:		 <start> eso pienso yo . <end>
Target sentence:		 <start> i do think so . <end>
Sampled sentence (T=0.01):	 <start> i think so . <end>
Sampled sentence (T=1.0):	 <start> i do think so . <end>
Sampled sentence (T=2.0):	 <start> i now help yet . <end>
----------------
Source sentence:		 <start> usted sabe algo . <end>
Target sentence:		 <start> you know something . <end>
Sampled sentence (T=0.01):	 <start> you know something . <end>
Sampled sentence (T=1.0):	 <start> you know something . <end>
Sampled sentence (T=2.0):	 <start> traveled check something . <end>
----------------
Source sentence:		 <start> puede que el cambie de idea . <end>
Target sentence:		 <start> he might change his mind . <end>
Sampled sentence (T=0.01):	 <start> he may return of her . <end>
Sampled sentence (T=1.0):	 <start> he may keep idea . <end>
Sampled sentence (T=2.0):	 <start> he sure like smile support . <end>
----------------
Source sentence:		 <start> estoy viviendo mi sueno . <end>
Target

## Beam Search Decoding

Here, instead of sampling sequences from $P(\mathbf{y}\mid \mathbf{x})$, I will find the maximum *a-posteriori* estimate $\mathbf{\hat y} = \mathrm{argmax}_{\mathbf{y}} P(\mathbf{y}\mid \mathbf{x})$. Because it is not possible to search over all sequences $\mathbf{y}$, we use a heuristic method called beam search decoding.

The idea is as follows: at each timestep, you keep track of the $K$ best hypotheses (in terms of probability) seen up to that time that have not terminated; all terminated hypotheses (those that have predicted the end-of-sentence token) that are encountered should be saved along with their log probability. Once you have proceeded for the maximum number of timesteps, the final hypothesis is the one among the terminated hypotheses that has highest probability.

However, to avoid bias toward shorter hypotheses, we apply a length penalty (described on page 12 of [this paper](https://arxiv.org/pdf/1609.08144)) when selecting the final hypothesis. Specifically, out of the hypotheses that have terminated, we select the one that maximizes the score $$\frac{\log P(\mathbf{y}\mid \mathbf{x})}{lp}$$ where the length penalty $lp$ is defined as $$lp = \Big (\frac{5+t}{5+1}\Big )^\alpha $$ The higher the hyperparameter $\alpha$ is, the more short hypotheses are penalized.

First implementing 3 helper functions

In [ ]:
def sort_predictions(predictions):
    '''
    Sort prediction probabilities in descending order, and keep track of which hypothesis and vocab index produced each.

    Args:
        predictions: A tensor of size [beam_size, trg_vocab_size] containing log probabilities.

    Returns:
        probs: A tensor of size beam_size*trg_vocab_size containing all log probabilities sorted in descending order. Each row correponds to a separate
           hypothesis.
        idxes: A tensor of size beam_size*trg_vocab_size, where idxes[i] indicates which index in the vocabulary probs[i] corresponds
           to. Thus, 0 ≤ idxes[i] < trg_vocab_size.
        hypothesis_ids: A tensor of size beam_size*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis probs[i]
           corresonds to. Thus, 0 ≤ hypothesis_ids[i] < beam_size.
    '''
    assert len(predictions.shape) == 2, 'predictions should be a 2d tensor'
    beam_size, trg_vocab_size = predictions.shape[0], predictions.shape[1]
    probs = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)
    idxes = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)
    hypothesis_ids = torch.zeros(beam_size * trg_vocab_size, device=predictions.device)

    flat_probs = predictions.view(-1)
    sorted_probs, sorted_indices = torch.sort(flat_probs, descending=True)

    probs = sorted_probs
    hypothesis_ids = sorted_indices // trg_vocab_size
    idxes = sorted_indices % trg_vocab_size

    return probs, idxes, hypothesis_ids

The cell below will provide a sanity check for this function.

In [ ]:

def sanityCheckSortPredictions():
    predictions_list = [torch.tensor([[-9.3672, -10.1368],[-5.2810, -7.4104],[-6.2810, -8.4154]]),
                        torch.tensor([[-5.2810, -7.4104],[-9.3672, -10.1368],[-6.2810, -8.4154]]),
                        torch.tensor([[-6.2810, -8.4154],[-5.2810, -7.4104],[-9.3672, -10.1368]]),
                        torch.tensor([[-10.4256, -4.9818],[-5.0244, -8.7471],[-7.2406, -6.3092]]),
                        torch.tensor([[-7.5414, -7.9009],[-8.2827, -8.5210],[-9.2406, -9.3092]])]
    expected_outputs = [(torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([1, 2, 1, 2, 0, 0])),
                        (torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([0, 2, 0, 2, 1, 1])),
                        (torch.tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368]),
                         torch.tensor([0, 0, 1, 1, 0, 1]), torch.tensor([1, 0, 1, 0, 2, 2])),
                        (torch.tensor([ -4.9818,  -5.0244,  -6.3092,  -7.2406,  -8.7471, -10.4256]),
                         torch.tensor([1, 0, 1, 0, 1, 0]), torch.tensor([0, 1, 2, 2, 1, 0])),
                        (torch.tensor([-7.5414, -7.9009, -8.2827, -8.5210, -9.2406, -9.3092]),
                         torch.tensor([0, 1, 0, 1, 0, 1]), torch.tensor([0, 0, 1, 1, 2, 2]))]

    print("--- TEST: sort_predictions() outputs ---")
    print()
    for i in range(len(predictions_list)):
        probs, idxes, hypotheses_ids = sort_predictions(predictions_list[i])
        if torch.equal(probs, expected_outputs[i][0]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected probs: ' + str(expected_outputs[i][0]) + '\tYour probs: '+ str(probs))
        print(message)

        if torch.equal(idxes, expected_outputs[i][1]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected idxes: ' + str(expected_outputs[i][1]) + '\tYour idxes: '+ str(idxes))
        print(message)

        if torch.equal(hypotheses_ids, expected_outputs[i][2]):
            status = "PASSED"
        else:
            status = "FAILED"
        message = '\t' + status + "\tInput: Tensor of shape " + str(predictions_list[i].shape) + ('\tExpected hypotheses_ids: ' + str(expected_outputs[i][2]) + '\tYour hypotheses_ids: '+ str(hypotheses_ids))
        print(message)
        print()

if __name__ == '__main__':
    sanityCheckSortPredictions()

--- TEST: sort_predictions() outputs ---

	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])	Your probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected idxes: tensor([0, 0, 1, 1, 0, 1])	Your idxes: tensor([0, 0, 1, 1, 0, 1])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected hypotheses_ids: tensor([1, 2, 1, 2, 0, 0])	Your hypotheses_ids: tensor([1, 2, 1, 2, 0, 0])

	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])	Your probs: tensor([ -5.2810,  -6.2810,  -7.4104,  -8.4154,  -9.3672, -10.1368])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected idxes: tensor([0, 0, 1, 1, 0, 1])	Your idxes: tensor([0, 0, 1, 1, 0, 1])
	PASSED	Input: Tensor of shape torch.Size([3, 2])	Expected hypotheses_ids: tensor([0, 2, 0, 2, 1, 1])	Your hypo

In [ ]:
def find_eos_sentences(probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, alpha):
    '''
    Find the sentences that have generated EOS, but only if they are within the top beam_size sentences ranked by log probability.
    Return the sentences (specified by a list of indexes from the vocabulary) that have terminated (generated EOS), and their scores.

    Args:
        probs: A tensor of size num_hyps*trg_vocab_size containing log probabilities (where 1 ≤ num_hyps ≤ beam_size). You may
            assume that this is sorted in descending order.
        idxes: A tensor of size num_hyps*trg_vocab_size where idxes[i] indicates which vocab index probs[i] corresponds to
        hypothesis_ids: A tensor of size num_hyps*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis probs[i] corresponds to
        curr_output: A tensor of size [num_hyps, t-1] containing vocab indexes chosen for each hypothesis up to time t-1.
        eos_idx: The end-of-sentence index in the vocabulary.
        beam_size: The beam size.
        alpha: The alpha used in calculating length penalty.
    Returns:
        eos_sents: A list of lists, where eos_sents[i] is a list containing the vocab indices of the ith sentence.
        eos_probs: A list where eos_probs[i] is the score of eos_sents[i]. The score is the log proabability of the sentence, divided by
            the length penalty, which is described above.
    '''

    assert probs.shape == idxes.shape == hypothesis_ids.shape, 'probs, idxes, and hypothesis_ids must all be the same shape'
    assert alpha >= 0 and eos_idx >= 0 and beam_size >= 1

    eos_probs, eos_sents = [], []


    #    1. Find the indexes of idxes that (1) correspond to predicting EOS and (2) are within the top beam_size results ranked by log probability.
    #         Hint: Make a binary mask that is the same size as probs, where an element is True if it meets these conditions.
    mask = (idxes == eos_idx)
    top_beam_mask = torch.zeros_like(mask)
    top_beam_mask[:beam_size] = True
    combined_mask = mask & top_beam_mask
    top_eos_indices = torch.nonzero(combined_mask).squeeze(-1)

    #    2. Calculate the length penalty (do not count BOS or EOS in the length!), and divide the log probabilities by it. These are the scores.
    for i in top_eos_indices:
        hyp_id = hypothesis_ids[i].item()

        # Count non-zero tokens in the hypothesis (excluding padding)
        tokens = curr_output[hyp_id].tolist()
        # Remove padding (zeros) from the end
        while tokens and tokens[-1] == 0:
            tokens.pop()

        # Calculate actual length (don't count BOS at the beginning)
        # BOS is usually token 2 in many NLP tasks
        length = len(tokens)
        if tokens and tokens[0] == 2:  # If BOS is at the beginning
            length -= 1

        # Length penalty formula: ((5 + len)/6)^alpha where len is the length excluding BOS and EOS
        length_penalty = ((5 + length)/6) ** alpha

        # Calculate score
        score = probs[i].item() / length_penalty

        # Create the full sentence by appending EOS to the current hypothesis
        final_sent = curr_output[hyp_id].tolist()
        # Remove padding zeros from the end
        while final_sent and final_sent[-1] == 0:
            final_sent.pop()
        # Add EOS token
        final_sent.append(eos_idx)

        eos_sents.append(final_sent)
        eos_probs.append(score)

    return eos_sents, eos_probs



In [ ]:
def find_non_eos_sentences(probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, t):
    '''
    Finds the top beam_size continuations based on their cumulative log probability.

    Args:
        probs: A tensor of size beam_size*trg_vocab_size containing log probabilities. You may assume that this is
           sorted in descending order.
        idxes: A tensor of size beam_size*trg_vocab_size where idxes[i] indicates which vocab index probs[i]
           corresponds to
        hypothesis_ids: A tensor of size beam_size*trg_vocab_size where hypothesis_ids[i] indicates which hypothesis
           probs[i] corresponds to
        curr_output: A tensor of size [beam_size, max_decode_len] containing vocab indexes chosen for each hypothesis.
        eos_idx: The end-of-sentence index in the vocabulary.
        beam_size: The beam size.
        t: The timestep currently being processed.
    Returns:
        new_output: A tensor of size [beam_size, max_decode_len], where the first t columns are filled with the
           vocab indexes found so far for each hypothesis chosen.
        new_probs: A tensor of size beam_size where new_probs[i] contains the cumulative log probability of the
           hypothesis new_output[i]
        next_hyps: A tensor of size beam_size where next_hyps[i] contains the hypothesis index from the previous
           timestep that the selected hypothesis new_output[i] continues. For example, if new_output[i] continues
           hypothesis 4 from curr_output (i.e. it continues curr_output[4]), then next_hyps[i] = 4.
    '''

    assert beam_size == curr_output.shape[0], 'curr_output must have beam_size rows'
    assert probs.shape == idxes.shape == hypothesis_ids.shape
    assert eos_idx >= 0 and beam_size >= 1 and t >= 1

    new_output = torch.zeros_like(curr_output, device=curr_output.device)
    new_probs, next_hyps = torch.zeros(beam_size, device=curr_output.device), torch.zeros(beam_size, device=curr_output.device)

    #    1. Find the vocab indexes (hint: in idxes) and probabilities (hint: in probs) of the top beam_size continuations
    #       that are not EOS, based on their log probabilities.
    mask = (idxes != eos_idx)
    top_indices = torch.nonzero(mask).squeeze(-1)[:beam_size]

    selected_probs = probs[top_indices]
    selected_vocab_idxs = idxes[top_indices]
    selected_hyp_ids = hypothesis_ids[top_indices]

    #    2. Find the hypothesis indexes that each of these continues (hint: in hypothesis_ids).
    new_probs[:] = selected_probs
    next_hyps[:] = selected_hyp_ids

    #    3. Set the first t columns of new_output to the existing hypotheses that you have chosen to continue
    for i in range(beam_size):
        hyp_id = selected_hyp_ids[i].long()
        # Copy all tokens from the selected hypothesis (up to and including position t-1)
        new_output[i, :t] = curr_output[hyp_id, :t]
        # Add the new token at position t
        new_output[i, t] = selected_vocab_idxs[i]

    return new_output, new_probs, next_hyps

Main beam search function

In [ ]:
def beam_decode_model(encoder, decoder, src, max_decode_len, beam_size,alpha):
    """
    Perform beam search for the target sentence for the source sentence in src based on the trained encoder and decoder.

    Args:
        encoder: Your RnnEncoder object
        decoder: Your RnnDecoder object
        src: [max_src_length, 1] the source sentence you wish to translate (note: to simplify this function, we do not batch this)
        max_decode_len: The maximum desired length (int) of your target translated sentences
        beam_size: The beam size
        alpha: The alpha in the length penalty formula

    Returns:
        sentence: A list containing vocab indexes of the best target sentence found.

    Pseudo-code:
    - Obtain encoder output and hidden state by encoding src sentences
    - Expand start-of-sentence token by taking the beam_search continuations that have highest probability.
    - For 2 ≤ t < max_decode_len:
        - Expand each of the previous beam_sarch hypotheses, and compute the cumulative log probability of each possible continuation
        - Save the hypotheses that terminate and their log probabilities if they are in the top beam_size hypotheses by log probability
        - Select the beam_size continuations that are not EOS to proceed with based on their log probabilities, and also track
          which hypothesis they continue
        - For the RNN only (not transformer), choose the hidden states correspoonding to the hypotheses that have been selected to continue
    - Among the terminated sentences, select the one with highest score and return it

    Hints:
    - You should interact with the decoder using its decode_step function.
    - This function will be used for the transformer model as well later in the homework. That model does not have a hidden state, so
      dec_hidden is None.
      Thus, when choosing which hidden state vectors to pass between timesteps, you only need to do this when the variable is_rnn is True.
    """

    # Initialize variables
    trg_vocab = decoder.trg_vocab
    eos_idx = trg_vocab.word2idx['<end>']
    batch_size = src.size(1)
    assert batch_size == 1 # For beam search, we will keep it simple and just do one item at a time.
    device = src.device
    is_rnn = 'Rnn' in encoder.__class__.__name__
    sentence = []

    # For beam search, we initialize our candidate hypotheses as start-of-sentence, each having probability 1 (log probability 0)
    curr_output = torch.zeros((beam_size, max_decode_len),device=device) # For beam search, have to track outputs for each beam
    curr_probs = torch.zeros(beam_size,device=device) # Each candidate hypothesis probability
    completed_hypotheses = []
    completed_scores = []

    # We start the decoding with the start token for each example
    dec_input = torch.tensor([[trg_vocab.word2idx['<start>']]] * batch_size, device=device)
    curr_output[:, 0] = dec_input.squeeze(1)

    # Obtain encoder outputs
    if is_rnn: enc_output, dec_hidden = encoder(src)
    else: enc_output, dec_hidden = encoder(src), None
    enc_output, dec_hidden = enc_output.to(device), dec_hidden.to(device) if dec_hidden is not None else None

    # Step 1: Decode start token
    logits, dec_hidden = decoder.decode_step(dec_input, enc_output, dec_hidden)
    log_probs = torch.log_softmax(logits, dim=-1)  # [1, vocab_size]

    # Step 2: Expand predictions and sort
    flat_probs, idxes, hyp_ids = sort_predictions(log_probs)  # each of shape [vocab_size]

    # Step 3: Select top beam_size non-EOS to initialize beam
    curr_output, curr_probs, next_hyps = find_non_eos_sentences(
        flat_probs, idxes, hyp_ids, curr_output, eos_idx, beam_size, t=1
    )
    enc_output = enc_output.repeat(1, beam_size, 1)
    if is_rnn:
        dec_hidden = dec_hidden.repeat(1, beam_size, 1)

    for t in range(2, max_decode_len):
        dec_input = curr_output[:, :t]
        logits, dec_hidden = decoder.decode_step(dec_input, enc_output, dec_hidden)
        log_probs = torch.log_softmax(logits, dim=-1)  # [beam_size, vocab_size]

        cumulative_log_probs = log_probs + curr_probs.unsqueeze(1)

        flat_probs, idxes, hypothesis_ids = sort_predictions(cumulative_log_probs)

        eos_sents, eos_scores = find_eos_sentences(flat_probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, alpha)
        completed_hypotheses.extend(eos_sents)
        completed_scores.extend(eos_scores)

        curr_output, curr_probs, next_hyps = find_non_eos_sentences(
            flat_probs, idxes, hypothesis_ids, curr_output, eos_idx, beam_size, t
        )

        if is_rnn:
            dec_hidden = dec_hidden[:, next_hyps.long(), :]

    if len(completed_hypotheses) > 0:
        best_idx = torch.tensor(completed_scores).argmax().item()
        sentence = completed_hypotheses[best_idx]
    else:
        best_idx = curr_probs.argmax().item()
        sentence = curr_output[best_idx][:max_decode_len].tolist()

        if eos_idx in sentence:
            sentence = sentence[:sentence.index(eos_idx) + 1]

    return sentence

Qualitatively compare some of the sentences generated by model with the some of the correct translations.

In [ ]:
if __name__ == '__main__':
    rnn_encoder.eval()
    rnn_decoder.eval()
    for h in range(5): # Do whole thing 5 times here, since doing batch size of 1 for beam search
        idxes = random.choices(range(len(test_dataset.dataset)), k=1)
        src, trg =  train_dataset.dataset[idxes]
        beam_size = 5
        alphas = [0.6, 100]
        beam_result = [beam_decode_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), beam_size,alpha=alpha) for alpha in alphas]
        greedy_result = beam_decode_model(rnn_encoder, rnn_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), 1,alpha=0)

        print("Source sentence:\t\t\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[0]] if x != '<pad>']))
        print("Target sentence:\t\t\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[0]] if x != '<pad>']))
        print("Predicted sentence (greedy search):\t\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in greedy_result] if x != '<pad>']))
        for i in range(len(alphas)):
            print("Predicted sentence (beam search, alpha="+str(alphas[i])+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in beam_result[i]] if x != '<pad>']))
        print("----------------")



Source sentence:				 <start> hizo llorar a mi madre . <end>
Target sentence:				 <start> it made my mother cry . <end>
Predicted sentence (greedy search):		 <start> he did my mother . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> he did my mother . <end>
Predicted sentence (beam search, alpha=100):	 <start> do you hit my mother cry . m on . <end>
----------------
Source sentence:				 <start> perdi el tren de las . <end>
Target sentence:				 <start> i missed the train . <end>
Predicted sentence (greedy search):		 <start> i missed the train . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> i missed the train . <end>
Predicted sentence (beam search, alpha=100):	 <start> i ve got on the train about . sorry . <end>
----------------
Source sentence:				 <start> no puedo protegerte aqui . <end>
Target sentence:				 <start> i can t protect you here . <end>
Predicted sentence (greedy search):		 <start> i can t protect you here . <end>
Predicted sentence (beam search, a

## Evaluate RNN Model

1.   https://en.wikipedia.org/wiki/BLEU
2.   https://www.aclweb.org/anthology/P02-1040.pdf

In [ ]:
def evaluate_model(encoder, decoder, test_dataset, target_tensor_val, device):
    trg_vocab = decoder.trg_vocab
    batch_size = test_dataset.batch_size
    pad_idx = trg_vocab.word2idx['<pad>']

    encoder.eval()
    decoder.eval()

    final_output, target_output = [], []

    with torch.no_grad():
        for batch, (src, trg) in enumerate(test_dataset):
            sentences = sample_model(encoder,
                                     decoder,
                                     src.transpose(0,1).to(DEVICE),
                                     trg.size(1),
                                     temperature=0.0000001) # Low temperature ==> greedy decoding

            final_output += sentences
            trg_sents = [trg[i,:][trg[i,:] != pad_idx].tolist() for i in range(trg.shape[0])]
            target_output += trg_sents

    # Compute BLEU scores
    return compute_bleu_scores(target_output, final_output, trg_vocab)

if __name__ == '__main__':
    rnn_save_candidate, rnn_scores = evaluate_model(rnn_encoder, rnn_decoder, test_dataset, trg_tensor_val, DEVICE)

BLEU 1-gram: 0.296868
BLEU 2-gram: 0.082743
BLEU 3-gram: 0.060334
BLEU 4-gram: 0.057294


# Step 4: Train a Transformer

Writing a transformer model for machine translation, training and evaluating its results.

In [ ]:
import math

## Positional Embeddings

Similar to the RNN, we start with the Encoder model. A key component of the encoder is the Positional Embedding. As we know, word embeddings encode words in such a way that words with similar meaning have similar vectors. Because there are no recurrences in a Transformer, we need a way to tell the transformer the relative position of words in a sentence: so will add a positional embedding to the word embeddings. Now, two words with a similar embedding will both be close in meaning and occur near each other in the sentence.

Creating a positional embedding matrix of size $(max\_len, embed\_dim)$ using the following formulae:
<br>
$\begin{align*} pe[pos,2i] &= \sin \Big (\frac{pos}{10000^{2i/embed\_dim}}\Big )\\pe[pos,2i+1] &= \cos \Big (\frac{pos}{10000^{2i/embed\_dim}}\Big ) \end{align*}$

In [ ]:
def create_positional_embedding(max_len, embed_dim):
    '''
    Args:
        max_len: The maximum length supported for positional embeddings
        embed_dim: The size of your embeddings
    Returns:
        pe: [max_len, 1, embed_dim] computed as in the formulae above
    '''
    pe = None

    position = torch.arange(0, max_len).unsqueeze(1).float()  # [max_len, 1]
    div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * -(math.log(10000.0) / embed_dim))  # [embed_dim/2]

    pe = torch.zeros(max_len, embed_dim)  # ✅ Fix: initialize first

    pe[:, 0::2] = torch.sin(position * div_term)   # sin for even indices
    pe[:, 1::2] = torch.cos(position * div_term)   # cos for odd indices

    pe = pe.unsqueeze(1)

    return pe

## Encoder Model

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, src_vocab, embedding_dim, num_heads,
        num_layers, dim_feedforward, max_len_src, device, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        self.device = device
        """
        Args:
            src_vocab: Vocab_Lang, the source vocabulary
            embedding_dim: the dimension of the embedding (also the number of expected features for the input of the Transformer)
            num_heads: The number of attention heads
            num_layers: the number of Transformer Encoder layers
            dim_feedforward: the dimension of the feedforward network models in the Transformer
            max_len_src: maximum length of the source sentences
            device: the working device (you may need to map your postional embedding to this device)
            dropout: the dropout to be applied. Default=0.1.
        """
        self.src_vocab = src_vocab # Do not change
        src_vocab_size = len(src_vocab)

        # Create positional embedding matrix
        self.position_embedding = create_positional_embedding(max_len_src, embedding_dim).to(device)
        self.register_buffer('positional_embedding', self.position_embedding) # this informs the model that position_embedding is not a learnable parameter


        # Initialize embedding layer
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Initialize a nn.TransformerEncoder model (you'll need to use embedding_dim,
        #    num_layers, num_heads, & dim_feedforward here)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    def make_src_mask(self, src):
        """
        Args:
            src: [max_len, batch_size]
        Returns:
            Boolean matrix of size [batch_size, max_len] indicating which indices are padding
        """
        assert len(src.shape) == 2, 'src must have exactly 2 dimensions'
        src_mask = src.transpose(0, 1) == 0 # padding idx
        return src_mask.to(self.device) # [batch_size, max_src_len]

    def forward(self, x):
        """
        Args:
            x: [max_len, batch_size]
        Returns:
            output: [max_len, batch_size, embed_dim]
        Pseudo-code (note: x refers to the original input to this function throughout the pseudo-code):
        - Pass x through the word embedding
        - Add positional embedding to the word embedding, then apply dropout
        - Call make_src_mask(x) to compute a mask: this tells us which indexes in x
          are padding, which we want to ignore for the self-attention
        - Call the encoder, with src_key_padding_mask = src_mask
        """
        output = None



        word_embed = self.embedding(x)
        pos_embed = self.position_embedding.to(x.device)[:x.size(0), :, :]
        x_embed = self.dropout(word_embed + pos_embed)

        src_mask = self.make_src_mask(x)
        src_mask = src_mask.to(x.device)

        output = self.encoder(x_embed, src_key_padding_mask=src_mask)

        return output

In [ ]:
if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    dimf = [50, 100, 150]
    embedding_dim = [4, 8, 12]
    max_len = 16
    num_layers = iter([1,1,1,2,2,2,3,3,3])
    nheads = iter([1, 1, 1, 1, 2, 2, 2, 4, 4])
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for df in dimf:
        for ed in embedding_dim:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = df
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inputs.append(inp)
    # Test init
    expected_outputs = [570, 1218, 1994, 2020, 4096, 6428, 4370, 8674, 13362]

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "init")

--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 4, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 570	Your Num. Params: 570
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 1218	Your Num. Params: 1218
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 12, 'num_heads': 1, 'dim_feedforward': 50, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 1994	Your Num. Params: 1994
	PASSED	Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 4, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 2, 'max_len_src': 16, 'device': device(type='cuda')}	Expected Num. Params: 2020	Your Num. Params: 2020
	PASSED	Input: {'src_vocab': ['a', 'a

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
if __name__=="__main__":
    # Set random seed
    torch.manual_seed(42)
    # Test forward
    inputs = []
    embedding_dims = [32,64,128]
    batch_sizes = [1, 2]
    dimf = 100
    nheads = iter([1,1,2,2,4,4])
    num_layers = iter([1,1,2,2,3,3])
    max_len = 16
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    for ed in embedding_dims:
        for b in batch_sizes:
            inp = {}
            inp['src_vocab'] = sanity_vocab
            inp['embedding_dim'] = ed
            inp['num_heads'] = next(nheads)
            inp['dim_feedforward'] = dimf
            inp['num_layers'] = next(num_layers)
            inp['max_len_src'] = max_len
            inp['device'] = DEVICE
            inp["batch_size"] = b
            inputs.append(inp)
    expected_outputs = [torch.Size([16, 1, 32]), torch.Size([16, 2, 32]), torch.Size([16, 1, 64]), torch.Size([16, 2, 64]), torch.Size([16, 1, 128]), torch.Size([16, 2, 128])]

    sanityCheckModel(inputs, TransformerEncoder, expected_outputs, "forward")

--- TEST: Output shape of forward(...) ---
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 32, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 1])	Expected Output Shape: torch.Size([16, 1, 32])	Your Output Shape: torch.Size([16, 1, 32])
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 32, 'num_heads': 1, 'dim_feedforward': 100, 'num_layers': 1, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 2])	Expected Output Shape: torch.Size([16, 2, 32])	Your Output Shape: torch.Size([16, 2, 32])
	PASSED	 Init Input: {'src_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 64, 'num_heads': 2, 'dim_feedforward': 100, 'num_layers': 2, 'max_len_src': 16, 'device': device(type='cuda')}	Forward Input Shape: torch.Size([16, 1])	Expected Output Shape: torch.Size([16, 1, 64])	Your Output Shape: torch.Size([16, 1, 64])
	PASSED	 Init I

##  Decoder Model

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, trg_vocab, embedding_dim, num_heads,
        num_layers, dim_feedforward, max_len_trg, device, dropout=0.1):
        super(TransformerDecoder, self).__init__()
        self.device = device
        """
        Args:
            trg_vocab: Vocab_Lang, the target vocabulary
            embedding_dim: the dimension of the embedding (also the number of expected features for the input of the Transformer)
            num_heads: The number of attention heads
            num_layers: the number of Transformer Decoder layers
            dim_feedforward: the dimension of the feedforward network models in the Transformer
            max_len_trg: maximum length of the target sentences
            device: the working device (you may need to map your postional embedding to this device)
            dropout: the dropout to be applied. Default=0.1.
        """
        self.trg_vocab = trg_vocab # Do not change
        trg_vocab_size = len(trg_vocab)

        # Create positional embedding matrix
        self.position_embedding = create_positional_embedding(max_len_trg, embedding_dim).to(device)
        self.register_buffer('positional_embedding', self.position_embedding) # this informs the model that positional_embedding is not a learnable parameter


        # Initialize embedding layer
        self.embedding = nn.Embedding(trg_vocab_size, embedding_dim)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

        # Initialize a nn.TransformerDecoder model (you'll need to use embedding_dim,
        # num_layers, num_heads, & dim_feedforward here)
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        # Final fully connected layer
        self.fc = nn.Linear(embedding_dim, trg_vocab_size)

    def generate_square_subsequent_mask(self, sz):
        """Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(self.device)
        return mask

    def forward(self, dec_in, enc_out):
        """
        Args:
            dec_in: [sequence length, batch_size]
            enc_out: [max_len_src, batch_size, embed_dim]
        Returns:
            output: [sequence length, batch_size, trg_vocab_size]
        Pseudo-code:
            - Compute input word and positional embeddings in similar manner to encoder
            - Call generate_square_subsequent_mask() to compute a mask: this time,
              the mask is to prevent the decoder from attending to tokens in the "future".
              In other words, at time step i, the decoder should only attend to tokens
              1 to i-1.
            - Call the decoder, with tgt_mask = trg_mask
            - Run the output through the fully-connected layer and return it
        """
        output = None


        word_embed = self.embedding(dec_in)
        pos_embed = self.position_embedding.to(dec_in.device)[:dec_in.size(0), :, :]
        dec_embed = self.dropout(word_embed + pos_embed)

        trg_mask = self.generate_square_subsequent_mask(dec_in.size(0))

        decoded = self.decoder(dec_embed, enc_out, tgt_mask=trg_mask)
        output = self.fc(decoded)

        return output

    def decode_step(self, inputs, enc_output, dec_hs):
        '''
        Call one step of the decoder.

        Args:
            inputs: Input tokens; [batch_size, sequence length]
            enc_output: Encoder outputs; [max_len_src, batch_size, embed_dim]
            dec_hs: None

        Returns:
            fc_out: (Unnormalized) output distribution [batch_size, vocab_size]
            dec_hs: None
        '''
        assert dec_hs is None, 'For the transformer model, make sure you pass dec_hs = None!'
        inputs = inputs.type(torch.long)
        preds = self(inputs.transpose(0,1), enc_output)[-1]
        return preds, None

In [ ]:
def sanityCheckTransformerDecoderModelForward(inputs, NN, expected_outputs):
    print('--- TEST: Output shape of forward(...) ---\n')
    msg = ''
    for i, inp in enumerate(inputs):
        input_rep = '{'
        for k,v in inp.items():
            if torch.is_tensor(v):
                input_rep += str(k) + ': ' + 'Tensor with shape ' + str(v.size()) + ', '
            else:
                input_rep += str(k) + ': ' + str(v) + ', '
        input_rep += '}'
        dec = NN(trg_vocab=inp['trg_vocab'],embedding_dim=inp['embedding_dim'],num_heads=inp['num_heads'],num_layers=inp['num_layers'],dim_feedforward=inp['dim_feedforward'],max_len_trg=inp['max_len_trg'],device=inp['device'])
        dec_in = torch.randint(low=0,high=len(inputs[0]['trg_vocab']),size=(inp['max_len_trg'], inp['batch_size']))
        enc_out = torch.rand(inp['max_len_trg'], inp['batch_size'], inp['embedding_dim'])
        inp['encoder_outputs'] = enc_out
        with torch.no_grad():
            stu_out = dec(enc_out=enc_out, dec_in=dec_in)
        del dec
        has_passed = True
        if not torch.is_tensor(stu_out):
            has_passed = False
            msg = 'Output must be a torch.Tensor; received ' + str(type(stu_out))
        status = 'PASSED' if has_passed else 'FAILED'
        if not has_passed:
            message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (dec_in): ' + str(dec_in.shape) + '\tExpected Output Shape: ' + str(expected_outputs[i]) + '\t' + msg
            print(message)
            continue

        has_passed = stu_out.size() == expected_outputs[i]
        msg = 'Your Output Shape: ' + str(stu_out.size())
        status = 'PASSED' if has_passed else 'FAILED'
        message = '\t' + status + "\t Init Input: " + input_rep + '\tForward Input Shape (dec_in): ' + str(dec_in.shape) + '\tExpected Output Shape: ' + str(expected_outputs[i]) + '\t' + msg
        print(message)



In [ ]:
if __name__ == '__main__':
    # Set random seed
    torch.manual_seed(42)
    # Create test inputs
    hidden_units = [50, 100, 200]
    embedding_dim = [8, 16]
    num_heads = [1, 2]
    dim_feedforward = [50, 100]
    num_layers = [1, 2]
    max_lens = 64
    sanity_vocab = Vocab_Lang(vocab=["a", "aa", "aaa"])
    params = []
    inputs = []
    for ed in embedding_dim:
        for df in dim_feedforward:
            for nh in num_heads:
                for nl in num_layers:
                    inp = {}
                    inp['trg_vocab'] = sanity_vocab
                    inp['embedding_dim'] = ed
                    inp['num_heads'] = nh
                    inp['num_layers'] = nl
                    inp['dim_feedforward'] = df
                    inp['max_len_trg'] = max_lens
                    inp['device'] = DEVICE
                    inputs.append(inp)
    # Test init
    expected_outputs = [1567, 3049, 1567, 3049, 2417, 4749, 2417, 4749]
    sanityCheckModel(inputs, TransformerDecoder, expected_outputs, "init")
    print()

    # Test forward
    inputs = []
    batch_sizes = [1, 2, 4]
    num_heads = 2
    num_layers = 1
    embedding_dims = iter([100, 100, 200, 200, 200, 400, 400, 800, 800])
    max_lens = iter([16, 16, 16, 32, 32, 32, 64, 64, 128])
    expected_outputs = [torch.Size([16, 1, 5]),torch.Size([16, 2, 5]),torch.Size([16, 4, 5]),torch.Size([32, 1, 5]),torch.Size([32, 2, 5]),torch.Size([32, 4, 5]),torch.Size([64, 1, 5]),torch.Size([64, 2, 5]),torch.Size([128, 4, 5])]

    for hu in hidden_units:
        for b in batch_sizes:
            inp = {}
            edim = next(embedding_dims)
            inp['embedding_dim'] = edim
            inp['trg_vocab'] = sanity_vocab
            inp['num_heads'] = num_heads
            inp['num_layers'] = num_layers
            inp["batch_size"] = b
            inp['dim_feedforward'] = hu
            inp['max_len_trg'] = next(max_lens)
            inp['device'] = DEVICE
            inputs.append(inp)

    sanityCheckTransformerDecoderModelForward(inputs, TransformerDecoder, expected_outputs)


--- TEST: Number of Model Parameters (tests __init__(...)) ---
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'num_layers': 1, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 1567	Your Num. Params: 1567
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 1, 'num_layers': 2, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 3049	Your Num. Params: 3049
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 2, 'num_layers': 1, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 1567	Your Num. Params: 1567
	PASSED	Input: {'trg_vocab': ['a', 'aa', 'aaa'], 'embedding_dim': 8, 'num_heads': 2, 'num_layers': 2, 'dim_feedforward': 50, 'max_len_trg': 64, 'device': device(type='cuda')}	Expected Num. Params: 3049	Your Num. Params: 3049
	PASSED	Input: {'trg_vocab': ['a', 'a

## Train Transformer Model

Like the RNN, we train the encoder and decoder using cross-entropy loss.

In [ ]:
def train_transformer_model(encoder, decoder, optimizer, device, n_epochs):
    encoder.train()
    decoder.train()
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    for epoch in range(n_epochs):
        start = time.time()
        losses = []

        for src, trg in tqdm(train_dataset):

            src = src.to(device).transpose(0,1) # [max_src_length, batch_size]
            trg = trg.to(device).transpose(0,1) # [max_trg_length, batch_size]

            enc_out = encoder(src)
            output = decoder(trg[:-1, :], enc_out)

            output = output.reshape(-1, output.shape[2])
            trg = trg[1:].reshape(-1)

            optimizer.zero_grad()

            loss = criterion(output, trg)
            losses.append(loss.item())

            loss.backward()

            # Clip to avoid exploding grading issues
            torch.nn.utils.clip_grad_norm_(encoder.parameters(), max_norm=1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(), max_norm=1)

            optimizer.step()

        mean_loss = sum(losses) / len(losses)
        print('Epoch:{:2d}/{}\t Loss:{:.4f} ({:.2f}s)'.format(epoch + 1, n_epochs, mean_loss, time.time() - start))


In [ ]:
if __name__ == '__main__':
    # HYPERPARAMETERS
    LEARNING_RATE = 0.001
    DIM_FEEDFORWARD=512
    N_EPOCHS=10
    N_HEADS=2
    N_LAYERS=2
    DROPOUT=0.1

    transformer_encoder = TransformerEncoder(src_vocab, EMBEDDING_DIM, N_HEADS,
                                 N_LAYERS,DIM_FEEDFORWARD,
                                 max_length_src, DEVICE, DROPOUT).to(DEVICE)
    transformer_decoder = TransformerDecoder(trg_vocab, EMBEDDING_DIM, N_HEADS,
                              N_LAYERS,DIM_FEEDFORWARD,
                              max_length_trg, DEVICE, DROPOUT).to(DEVICE)

    transformer_model_params = list(transformer_encoder.parameters()) + list(transformer_decoder.parameters())
    optimizer = torch.optim.Adam(transformer_model_params, lr=LEARNING_RATE)

    print('Encoder and Decoder models initialized!')

Encoder and Decoder models initialized!


In [ ]:
if __name__ == '__main__':
    train_transformer_model(transformer_encoder, transformer_decoder, optimizer, DEVICE, N_EPOCHS)

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 1/10	 Loss:2.9319 (9.89s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 2/10	 Loss:1.8498 (10.12s)


  0%|          | 0/468 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss:0.6674 (10.20s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 9/10	 Loss:0.6175 (10.17s)


  0%|          | 0/468 [00:00<?, ?it/s]

Epoch:10/10	 Loss:0.5740 (10.03s)


## Inference

Now that we have trained the model, we can use it on test data.

In [ ]:
if __name__ == '__main__':
    transformer_encoder.eval()
    transformer_decoder.eval()
    idxes = random.choices(range(len(test_dataset.dataset)), k=5)
    src, trg =  train_dataset.dataset[idxes]
    results = {}
    for temp in [0.01, 1.0, 2.0]:
        results[temp] = sample_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), temp)
    for i in range(len(src)):
        print("Source sentence:\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[i]] if x != '<pad>']))
        print("Target sentence:\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[i]] if x != '<pad>']))
        for temp in results:
            print("Sampled sentence (T="+str(temp)+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in results[temp][i]] if x != '<pad>']))
        print("----------------")

Source sentence:		 <start> ¿ eso es un murcielago ? <end>
Target sentence:		 <start> is that a bat ? <end>
Sampled sentence (T=0.01):	 <start> is that a bat ? <end>
Sampled sentence (T=1.0):	 <start> is that a bat ? <end>
Sampled sentence (T=2.0):	 <start> is . that have a bat ? <end>
----------------
Source sentence:		 <start> quiero que se vaya tom . <end>
Target sentence:		 <start> i want tom to leave . <end>
Sampled sentence (T=0.01):	 <start> i want tom to go . <end>
Sampled sentence (T=1.0):	 <start> i want you to go . <end>
Sampled sentence (T=2.0):	 <start> i want wish you is sleepy . <end>
----------------
Source sentence:		 <start> sobre la mesa habia un gato . <end>
Target sentence:		 <start> a cat was on the table . <end>
Sampled sentence (T=0.01):	 <start> i was on the cat . <end>
Sampled sentence (T=1.0):	 <start> i took my cat . <end>
Sampled sentence (T=2.0):	 <start> tulips swimmer . <end>
----------------
Source sentence:		 <start> tom renuncio al plan . <end>
Target 

In [ ]:
if __name__ == '__main__':
    transformer_encoder.eval()
    transformer_decoder.eval()
    for h in range(5): # Do whole thing 5 times here, since doing batch size of 1 for beam search
        idxes = random.choices(range(len(test_dataset.dataset)), k=1)
        src, trg =  train_dataset.dataset[idxes]
        beam_size = 5
        alphas = [0.6, 100]
        beam_result = [beam_decode_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), beam_size,alpha=alpha) for alpha in alphas]
        greedy_result = beam_decode_model(transformer_encoder, transformer_decoder, src.transpose(0,1).to(DEVICE), trg.size(1), 1,alpha=0)

        print("Source sentence:\t\t\t\t", ' '.join([x for x in [src_vocab.idx2word[j.item()] for j in src[0]] if x != '<pad>']))
        print("Target sentence:\t\t\t\t", ' '.join([x for x in [trg_vocab.idx2word[j.item()] for j in trg[0]] if x != '<pad>']))
        print("Predicted sentence (greedy search):\t\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in greedy_result] if x != '<pad>']))
        for i in range(len(alphas)):
            print("Predicted sentence (beam search, alpha="+str(alphas[i])+"):\t", ' '.join([x for x in [trg_vocab.idx2word[j] for j in beam_result[i]] if x != '<pad>']))
        print("----------------")

Source sentence:				 <start> el no ha estado aqui mucho tiempo . <end>
Target sentence:				 <start> he hasn t been here long . <end>
Predicted sentence (greedy search):		 <start> he hasn t been here long . <end>
Predicted sentence (beam search, alpha=0.6):	 <start> he hasn t been here long . <end>
Predicted sentence (beam search, alpha=100):	 <start> the weather hasn t been here long long . . <end>
----------------
Source sentence:				 <start> ¿ lo puedo hacer enseguida ? <end>
Target sentence:				 <start> may i do it right now ? <end>
Predicted sentence (greedy search):		 <start> can i be done it ? <end>
Predicted sentence (beam search, alpha=0.6):	 <start> can i be done it ? <end>
Predicted sentence (beam search, alpha=100):	 <start> can i be done it now ? sorry for ? <end>
----------------
Source sentence:				 <start> hemos estado alli . <end>
Target sentence:				 <start> we ve been there . <end>
Predicted sentence (greedy search):		 <start> we ve been there . <end>
Predicted senten

## Evaluate Transformer Model

Now we can run the test set through the transformer model.

In [ ]:
if __name__ == '__main__':
    transformer_save_candidate, transformer_scores = evaluate_model(transformer_encoder, transformer_decoder, test_dataset, trg_tensor_val, DEVICE)

BLEU 1-gram: 0.300730
BLEU 2-gram: 0.084633
BLEU 3-gram: 0.062233
BLEU 4-gram: 0.059465
